# Responsible ML - Differential Privacy with SmartNoise 

## Install prerequisites

Before running the notebook, make sure the correct versions of these libraries are installed.

In [ ]:
!pip install opendp-smartnoise

In [ ]:
!pip install z3-solver

In [ ]:
!pip install pandas --upgrade

In [ ]:
!conda install protobuf -y

## Setup working directory

The cell below creates our working directory. This will hold our generated scripts.

In [ ]:
project_folder = './scripts'

if not os.path.exists(project_folder):
    os.makedirs(project_folder)

## Update Workspace details in script below

In the next cell, we update a Workspace config object using the `<subscription_id>`, `<resource_group_name>`, and `<workspace_name>`. This will fetch the matching Workspace and prompt you for authentication. Please click on the link and input the provided details.

For more information on **Workspace**, please visit: [Microsoft Workspace Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py)

`<subscription_id>` = You can get this ID from the landing page of your Resource Group.

`<resource_group_name>` = This is the name of your Resource Group.

`<workspace_name>` = This is the name of your Workspace.

In [ ]:
%%writefile $project_folder/utils.py
import pandas as pd
from azureml.core.workspace import Workspace
from azureml.core import Dataset
from azureml.data.datapath import DataPath
import z3
import reconstruction_module as rec
import numpy as np

def get_data():
    df = pd.read_parquet('creditdataset.parquet')
    return df

def fetch_data():
    df = pd.read_parquet('creditdataset.parquet')
    df = df.sample(n=500)
    df = df.select_dtypes(include='int')
    df = df.iloc[:,2:12]
    return df

def register_privatized_dataset(synth_df):
    ## 
    ## Update workspace details
    ##
    ws = Workspace(
        subscription_id = '<subscription_id>', 
        resource_group = '<resource_group>', 
        workspace_name = '<workspace_name>')
    ws.write_config()
    datastore = ws.get_default_datastore()
    synth_df.to_csv('privatized_ds.csv',index=False)
    datastore.upload_files(files=['privatized_ds.csv'], overwrite=True)
    datastore_path = [DataPath(datastore, 'privatized_ds.csv')]
    tabular = Dataset.Tabular.from_delimited_files(path=datastore_path)
    tabular = tabular.register(workspace=ws, 
                           name='privatized_ds',
                           description='Privatized training data',
                           create_new_version=True)
    tabular = Dataset.get_by_name(ws, 'privatized_ds')
    print(tabular.version)

def percentage(part, whole):
    return 100 * float(part)/float(whole)

def endcode_solver_data():
    rawDataset = pd.read_parquet('creditdataset.parquet')
    rawDataset.to_csv('creditdataset.csv', index=False)

    # load data 
    orig_data, data = rec.load_data()
    non_income_data = data.drop('INCOME', axis = 1)

    # get plausible variable combinations and subset of length 5 plausible combinations 
    plausible_variable_combinations = rec.get_plausible_variable_combinations(non_income_data)
    plausible_variable_combinations_names = ['__'.join(combination) for combination in plausible_variable_combinations]

    five_way_interactions = [combination for combination in plausible_variable_combinations if len(combination) == 5]
    five_way_interactions_names = ['__'.join(combination) for combination in five_way_interactions]

    # get dictionaries of private and non-private releases (up to 5-way interactions)
    count_dict, priv_count_dict, mean_income_dict, priv_mean_income_dict, median_income_dict, priv_median_income_dict, min_income_dict, priv_min_income_dict, max_income_dict, priv_max_income_dict = rec.create_dicts(data, non_income_data, plausible_variable_combinations)

    # get string representations of each element associated with each tuple representing the 5-way interactions
    elem_dict, priv_elem_dict = rec.create_elem_dicts(count_dict, priv_count_dict, five_way_interactions, five_way_interactions_names)
    
    # set applications
    applications, priv_applications = rec.get_applications(five_way_interactions, five_way_interactions_names,
                                                    plausible_variable_combinations, plausible_variable_combinations_names,
                                                    count_dict, priv_count_dict, 
                                                    mean_income_dict, priv_mean_income_dict,
                                                    median_income_dict, priv_median_income_dict,
                                                    min_income_dict, priv_min_income_dict,
                                                    max_income_dict, priv_max_income_dict,
                                                    elem_dict, priv_elem_dict, lowest_allowable_count = 1,
                                                    use_medians = True, use_mins = False, use_maxes = False)
    # remove duplicate applications
    applications = list(set(applications))
    priv_applications = list(set(priv_applications))
    
    # set applications
    applications_3, priv_applications_3 = rec.get_applications(five_way_interactions, five_way_interactions_names,
                                                plausible_variable_combinations, plausible_variable_combinations_names,
                                                count_dict, priv_count_dict, 
                                                mean_income_dict, priv_mean_income_dict,
                                                median_income_dict, priv_median_income_dict,
                                                min_income_dict, priv_min_income_dict,
                                                max_income_dict, priv_max_income_dict,
                                                elem_dict, priv_elem_dict, lowest_allowable_count = 10,
                                                use_medians = True, use_mins = False, use_maxes = False)
    
    return orig_data, applications_3, priv_applications_3, count_dict, elem_dict

### ---- Restart Kernel ----

## Data Exploration

Let's take a look at the different trends and distribuitions we are able to find in the open dataset.

### Distribution of income across different Age Groups

Plot distribuition of income accross different age groups.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from scripts.utils import *
warnings.filterwarnings('ignore')

df = get_data()
df = df.sort_values(by=['AGE_GROUP'])
sns.set(style="ticks")
f, ax = plt.subplots(figsize=(14.5, 6.5))
ax.set_xscale("log")

sns.boxplot(x="INCOME", y="AGE_GROUP", data=df,
            whis=[0, 100], palette="vlag")


sns.swarmplot(x="INCOME", y="AGE_GROUP", data=df,
              size=2, color=".3", linewidth=0)

ax.xaxis.grid(True)
ax.set(ylabel="")
sns.despine(trim=True, left=True)
plt.title('Distribution of income across different Age Groups')

### Suggested Approvals by Income and Age Group

Let’s analyze how our dataset approval suggestions are distributed across the different incomes and age groups.

In [ ]:
f, ax = plt.subplots(figsize=(14.5, 6.5))
# Draw a nested violinplot and split the violins for easier comparison
sns.violinplot(x="AGE_GROUP", y="INCOME", hue="SHOULD_APPROVE",
               split=True, inner="quart",
               data=df, ax=ax)
sns.despine(left=True)
plt.title('Suggested Approvals by Income and Age Group')

### Employment Length Scatter Plot

We are also able to take a look at the different employment lengths that we have across our current dataset and group it by city and income ranges.

In [ ]:
sns.set(style="whitegrid")

f, ax = plt.subplots(figsize=(14.5, 6.5))
sns.despine(f, left=True, bottom=True)
sns.scatterplot(x="AGE", y="EMPLOYMENT_LENGTH",
                hue="CITY", size="INCOME",
                palette="ch:r=-.2,d=.3_r",
                sizes=(10, 400), linewidth=0,
                data=df, ax=ax)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('Employment Length Scatter Plot')

### Requests Heatmap

Analyze how many requests are contained in our dataset and to which city and month they belong to.

In [ ]:
import datetime

df['APPLICATION_DATE'] =  pd.to_datetime(df['APPLICATION_DATE'], format='%m/%d/%Y')
heatmap = df[['APPLICANT_ID','CITY','APPLICATION_DATE']]
heatmap['MONTH'] = df['APPLICATION_DATE'].dt.strftime("%B")
heatmap = heatmap.drop(columns=['APPLICATION_DATE'])
heatmap = pd.DataFrame({'APPLICANTS' : heatmap.groupby(['MONTH','CITY']).size()}).reset_index()
heatmap = heatmap.pivot("MONTH", "CITY", "APPLICANTS")
heatmap = heatmap.dropna()
heatmap
f, ax = plt.subplots(figsize=(14.5, 6.5))
sns.heatmap(heatmap, annot=True, linewidths=.5, ax=ax)
plt.title('Requests Heatmap')

### Employment Length by Age

The dataset also contains data that will allow us to see the different employment lengths that where recorded at the time that the application was submitted.

In [ ]:
import numpy as np
sns.set(style="ticks")

sns.jointplot(df['AGE'], df['EMPLOYMENT_LENGTH'], kind="hex", color="#4CB391", height=8.5)

# Differential Privacy

Differential Privacy is a technology that enables researchers and analysts to extract useful answers from
databases containing personal information and, at the same time, offers strong individual privacy
protections.

## Advanced Dataset Reconstruction Attack

The following cells will walk us through a dataset reconstruction attack in which we will attempt to reconstruct the non-private dataset based on its statistical features.

### Income Analysis

Income distribuition for individuals with a `DEGREE`.

In [ ]:
import z3
import reconstruction_module as rec
import seaborn as sns
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10]

orig_data, applications_3, priv_applications_3, count_dict, elem_dict = endcode_solver_data()

# define sample of interest
sample_of_interest = orig_data.loc[orig_data['DEGREE'] == 1]


# define bin edges
hist_edges = list(range(0, 110000, 10000))


# get non-private histogram values
binned_income = pd.cut(sample_of_interest['INCOME'], hist_edges, right = False)
binned_income = binned_income.cat.add_categories('>= 100,000')
binned_income.fillna('>= 100,000', inplace = True)
non_priv_hist = list(binned_income.value_counts().sort_index())

df = pd.DataFrame({'income': [str(cat) for cat in list(binned_income.cat.categories)] * 1,
                   'count': non_priv_hist,
                   'version': ['non-private'] * len(non_priv_hist) })

sns.barplot(x = 'income', y = 'count', hue = 'version', data = df)
plt.xticks(rotation = 45)
plt.title('Distribution of income for individuals with a degree')


### Reconstruction Attack

Imagine that the attacker has access to the following information outside of the statistical releases:

1. There is at least one person in the data with `DEGREE == 1`, `EMPLOYED == 2`, `agebinned == [45, 50)`, `SEX == 0`, and `MARRIED == 1` with an income of 95,000.
1. There is only one person in the data with `DEGREE == 1`, `EMPLOYED == 2`, `agebinned == [45, 50)`, `SEX == 1`, and `MARRIED == 1` and they have an income of 31,000.

In [ ]:
# remove duplicate applications
applications_3 = list(set(applications_3))
priv_applications_3 = list(set(priv_applications_3))

# initialize solvers
solver_3, solver_list_3 = rec.applications_to_solver(applications_3)
priv_solver_3, priv_solver_list_3 = rec.applications_to_solver(priv_applications_3) 

# add applications encoding existing attacker knowledge
group_1_def = 'DEGREE_1__EMPLOYED_1__agebinned_45,50__SEX_0__MARRIED_1'
group_1_elems = [z3.Int('{0}_{1}'.format(group_1_def, i)) for i in range(count_dict[group_1_def])]
solver_3.add(z3.Or([elem == 95_000 for elem in group_1_elems]))

group_2_def = 'DEGREE_1__EMPLOYED_1__agebinned_45,50__SEX_1__MARRIED_1'
solver_3.add(z3.Int( '{0}_{1}'.format(group_2_def, 0)) == 31_000)

# get results (models)
model_3 = rec.check_solution(solver_3) 
if model_3:
    print('non-private: Satisfied')
else:
    print('non-private: Unsatisfied')

# attempt to resconstruct data
recon_data_3 = rec.reconstruct_data(model_3, elem_dict)

# compare original and reconstructed data
orig_data, recon_data_3, exact_3, within_2k_3, within_5k_3 = rec.compare_data(orig_data, recon_data_3)

print('Of 500 total incomes:')
print('    {0} % of incomes reconstructed exactly'.format(percentage(exact_3, 500)))
print('    {0} % of incomes resconstructed within $2,000'.format(percentage(within_2k_3, 500)))
print('    {0} % of incomes resconstructed within $5,000'.format(percentage(within_5k_3, 500)))

### Reconstructed records plot with non-private data

The following plot shows the amount of records that were exactly reconstructed and reconstructed within the range of 2,000 and 5,000.

In [ ]:
df = pd.DataFrame({ '% of Reconstructed records' : [percentage(exact_3, 500), percentage(within_2k_3, 500), percentage(within_5k_3, 500)],
                  'Range': [ 'Exactly', 'Within $2,000', 'Within $5,000']})

sns.barplot(x = 'Range', y = '% of Reconstructed records', data = df)
plt.xticks(rotation = 45)
plt.title('% of Reconstructed records by range')

## Protecting Privacy with SmartNoise

The SmartNoise tools allow researchers and analysts to:

1. Use SQL dialect to create differentially private results over tabular data stores
2. Host a service to compose queries from heterogeneous differential privacy modules (including non-SQL) against shared privacy budget
2. Perform black-box stochastic testing against differential privacy modules

The SmartNoise system is currently aimed at scenarios where the researcher is trusted by the data owner.

<img align="left" src="./images/RespML-DifferentialPrivacy.gif"/>

### Apply SmartNoise Analysis

In [ ]:
import opendp.smartnoise.core as sn
    
with sn.Analysis(protect_floating_point=False) as analysis:
    data = sn.Dataset(value = list(sample_of_interest['INCOME']), column_names=['INCOME'])
    data = sn.resize(sn.to_int(data, lower = 0, upper = 100_000), number_columns = 1, lower = 0, upper = 100_000)
    dp_histogram = sn.dp_histogram(
        data,
        edges = hist_edges,
        upper = 500,
        null_value = 1_000_000,
        privacy_usage = {'epsilon': 1.0}
    )
analysis.release()
priv_hist = list(dp_histogram.value)    

# post-process dp histogram
priv_hist = [max(0, count) for count in priv_hist]

### Attack again

In [ ]:
# get results (models)
priv_model_3 = rec.check_solution(priv_solver_3) 
if priv_model_3:
    print('private: Unsatisfied')
else:
    print('private: Satisfied')
    exact_3 = 0
    within_2k_3 = 0
    within_5k_3 = 0

### Reconstructed records plot with private data

The following plot shows the amount of records that were exactly reconstructed and reconstructed within the range of 2,000 and 5,000.

In [ ]:
df = pd.DataFrame({ '% of Reconstructed records' : [percentage(exact_3, 500), percentage(within_2k_3, 500), percentage(within_5k_3, 500)],
                  'Range': [ 'Exactly', 'Within $2,000', 'Within $5,000']})

sns.barplot(x = 'Range', y = '% of Reconstructed records', data = df)
plt.xticks(rotation = 45)
plt.title('% of Reconstructed records by range')

## Differential Privacy Impact

The following plot demonstrated that although the data was privatized and injected with noise, its distributions stayed very similar having a non-destructive effect over the data usage.

In [ ]:
# create data frame for plotting
df = pd.DataFrame({'income': [str(cat) for cat in list(binned_income.cat.categories)] * 2,
                   'count': non_priv_hist + priv_hist,
                   'version': ['non-private'] * len(non_priv_hist) + ['private'] * len(priv_hist)})

df

sns.barplot(x = 'income', y = 'count', hue = 'version', data = df)
plt.xticks(rotation = 45)
plt.title('Distribution of income for individuals with a degree')

## Privatize Dataset

Now that we have explained the benefits of Differential Privacy with SmartNoise, lets privatize our training dataset and register it in our Azure Machine Learning workspace in order for us to leverage it for training a Machine Learning model.

### Non-private dataset

In [ ]:
from opendp.smartnoise.synthesizers.mwem import MWEMSynthesizer

df = fetch_data()
df

### Private dataset

The cell below contains code to privatize our dataset. `MWEMSynthesizer` will privatize our dataset with `epsilon = 3`.

In [ ]:
nf = df.to_numpy()

#Privatize dataset
synth = MWEMSynthesizer(epsilon=3, splits=[[0,1,2],[3,4],[5,6,7],[8],[9]])
synth.fit(nf)

sample_size = 500
synthetic = synth.sample(int(sample_size))
synth_df = pd.DataFrame(synthetic, 
    index=df.index,
    columns=df.columns)

register_privatized_dataset(synth_df)
synth_df